<center> <h1><bold>
<hr style="border-width:2px;border-color:#1664c8">
Exploration de la reconnaissance vocale et de la traduction en temps réel"</b><br>

<hr style="border-width:2px;border-color:#1664c8">
</center> </h1></bold>

Étape 1 : Configuration de l'environnement

In [1]:
# !pip install transformers torch sounddevice

     ---------------------------------------- 0.0/199.7 kB ? eta -:--:--
     ------------------------------------- 199.7/199.7 kB 11.8 MB/s eta 0:00:00


Étape 2 : Importation des bibliothèques

In [3]:
import numpy as np
import sounddevice as sd
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer, Wav2Vec2Processor, pipeline, MarianMTModel, MarianTokenizer


Étape 3 : Chargement du modèle Whisper

In [2]:
# Charger le modèle Whisper pour la reconnaissance vocale
whisper_model_name = "facebook/wav2vec2-base-960h"
tokenizer = Wav2Vec2Processor.from_pretrained(whisper_model_name)   # Wav2Vec2Tokenizer.from_pretrained(whisper_model_name)
model = Wav2Vec2ForCTC.from_pretrained(whisper_model_name)


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Étape 4 : Fonction de reconnaissance vocale en temps réel

In [34]:
def preprocess_audio(audio):
    # Normaliser les valeurs audio à l'intervalle [-1, 1]
    audio = ((audio / np.max(np.abs(audio)))+1)/2
    
    # Convertir l'audio en mono (si ce n'est pas déjà fait)
    if audio.shape[1] > 1:
        audio = np.mean(audio, axis=1)
    
    # Rééchantillonner l'audio à 16 kHz si nécessaire
    if len(audio) != 16000 * 5:  # Assurez-vous que l'audio a une durée de 5 secondes à 16 kHz
        audio = sd.resample(audio, 16000 * 5)
        
    audio = audio.reshape((80000,)) #.astype(np.int16)
    
    # Conversion en tensor PyTorch
    # inputs = tokenizer(audio, return_tensors="pt", padding="longest", return_attention_mask=True)
    return audio # inputs


def transcribe_audio(audio, tokenizer, model):
    inputs = tokenizer(audio, return_tensors="pt", padding="longest", return_attention_mask=True)
    print("inputs ",inputs)
    # with torch.no_grad():
    #     logits = model(input_values=inputs.input_values).logits
    logits = model(input_values=inputs.input_values).logits
    print("logits ", logits)
    predicted_ids = torch.argmax(logits, dim=-1)
    print("predicted_ids ", predicted_ids)
    transcription = tokenizer.batch_decode(predicted_ids)
    return transcription[0]


Étape 5 : Fonction de traduction

In [9]:
def translate_text(text, source_lang, target_lang):
    # Charger le modèle de traduction MarianMT
    model_name = f'Helsinki-NLP/opus-mt-fr-en'
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)
    
    # Tokeniser le texte
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    
    # Traduire le texte
    translation = model.generate(**inputs)
    translated_text = tokenizer.batch_decode(translation, skip_special_tokens=True)
    return translated_text[0]


Étape 6 : Reconnaissance vocale en temps réel et traduction

In [7]:
# def main():
source_language = "fr"  # Changer en fonction de la langue détectée
target_language = "en"  # Changer en fonction de la langue de traduction souhaitée

print("Démarrage de la reconnaissance vocale en temps réel...")
while True:
    try:
        audio_input = sd.rec(frames=int(16000 * 5), samplerate=16000, channels=1, dtype="int16")
        
        inputs = audio_input #preprocess_audio(audio_input)

        sd.wait()
        # text = transcribe_audio(inputs, tokenizer, model)
        text='Bonjour'
        print(f"Texte reconnu : {text}")

        # Traduction du texte
        translated_text = translate_text(text, source_language, target_language)
        print(f"Traduction : {translated_text}")

    except KeyboardInterrupt:
        print("Arrêt de la reconnaissance vocale en temps réel.")
        break

# if __name__ == "__main__":
#     main()


Démarrage de la reconnaissance vocale en temps réel...
Texte reconnu : Bonjour


C:\Users\olivi\anaconda3\envs\DataScientest\lib\site-packages\transformers\generation\utils.py:1346: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Traduction : Hello.
Texte reconnu : Bonjour
Traduction : Hello.
Texte reconnu : Bonjour
Traduction : Hello.
Texte reconnu : Bonjour
Traduction : Hello.
Texte reconnu : Bonjour
Traduction : Hello.
Texte reconnu : Bonjour
Traduction : Hello.
Texte reconnu : Bonjour
Traduction : Hello.
Arrêt de la reconnaissance vocale en temps réel.


In [48]:
inputs
audio_input.reshape((80000,)).astype(np.int16)

array([-3,  6,  9, ..., 10, 11,  7], dtype=int16)

In [25]:
audio_input = sd.rec(frames=int(16000 * 5), samplerate=16000, channels=1, dtype="int16")
audio_input = preprocess_audio(audio_input)
audio_input

array([ 0.50050356,  0.48841823,  0.01489303, ...,  0.48670919,
        0.3836787 , -0.50608844])

In [75]:
print(sampling_rate, len(audio_input), type(audio_input),type(audio_input[0][0]) )
audio_input.reshape(80000,)

16000 80000 <class 'numpy.ndarray'> <class 'numpy.float32'>


array([ 0.0000000e+00,  0.0000000e+00, -3.0517578e-05, ...,
       -9.1552734e-05,  0.0000000e+00,  9.1552734e-05], dtype=float32)

In [68]:
print(sampling_rate, len(dataset[0]["audio"]["array"]), type(dataset[0]["audio"]["array"]), type(dataset[0]["audio"]["array"][0]))
dataset[0]["audio"]["array"]

16000 93680 <class 'numpy.ndarray'> <class 'numpy.float64'>


array([0.00238037, 0.0020752 , 0.00198364, ..., 0.00042725, 0.00057983,
       0.0010376 ])

In [10]:
from transformers import AutoProcessor, Wav2Vec2ForCTC
from datasets import load_dataset
import torch

source_language = "fr"  # Changer en fonction de la langue détectée
target_language = "en"  # Changer en fonction de la langue de traduction souhaitée

 # load model and tokenizer
processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base-960h") # AutoProcessor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
sampling_rate = 16000
# language_detection = pipeline('text-classification',model="papluca/xlm-roberta-base-language-detection")

print("Démarrage de la reconnaissance vocale en temps réel...")
while True:
    try:

        audio_input = sd.rec(frames=int(16000 * 5), samplerate=16000, channels=1)
        sd.wait()
        # language = language_detection(transcribe_audio(audio_input, processor, model))[0]["language"]
        # print(f"Langue détectée : {language}")
        inputs = processor(audio_input.reshape(80000,), sampling_rate=sampling_rate, return_tensors="pt")
        sd.play(audio_input.reshape(80000,), 16000)
        with torch.no_grad():
            logits = model(**inputs).logits
        predicted_ids = torch.argmax(logits, dim=-1)

        # transcribe speech
        transcription = processor.batch_decode(predicted_ids)
        print("Transcription: ",transcription[0])
        # 'MISTER QUILTER IS THE APOSTLE OF THE MIDDLE CLASSES AND WE ARE GLAD TO WELCOME HIS GOSPEL'

        inputs["labels"] = processor(text=transcription[0], return_tensors="pt").input_ids

        # compute loss
        loss = model(**inputs).loss
        print("Loss: ",round(loss.item(), 2))
        
        sd.wait()
        text = transcription[0]
        print(f"Texte reconnu : {text}")

        # Traduction du texte
        translated_text = translate_text(text, source_language, target_language)
        print(f"Traduction : {translated_text}")

    except KeyboardInterrupt:
        print("Arrêt de la reconnaissance vocale en temps réel.")
        break

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Démarrage de la reconnaissance vocale en temps réel...
Transcription:  AN ON MY MEMMI LETDY GOCONOIS ON I
Loss:  30.59
Texte reconnu : AN ON MY MEMMI LETDY GOCONOIS ON I


C:\Users\olivi\anaconda3\envs\DataScientest\lib\site-packages\transformers\generation\utils.py:1346: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Traduction : AN ON MY MEMOMI LETDY GOCONOIS ON I
Transcription:  TO
Loss:  7.58
Texte reconnu : TO
Traduction : TO
Transcription:  ITI
Loss:  12.54
Texte reconnu : ITI
Arrêt de la reconnaissance vocale en temps réel.


In [5]:
from transformers import AutoProcessor, Wav2Vec2ForCTC
from datasets import load_dataset
import torch
      
dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
dataset = dataset.sort("id")
sampling_rate = dataset.features["audio"].sampling_rate
      
processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
      
# audio file is decoded on the fly
inputs = processor(dataset[0]["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits
predicted_ids = torch.argmax(logits, dim=-1)
       
 # transcribe speech
transcription = processor.batch_decode(predicted_ids)
print(transcription[0])
# 'MISTER QUILTER IS THE APOSTLE OF THE MIDDLE CLASSES AND WE ARE GLAD TO WELCOME HIS GOSPEL'
      
inputs["labels"] = processor(text=dataset[0]["text"], return_tensors="pt").input_ids
      
# compute loss
loss = model(**inputs).loss
round(loss.item(), 2)

Found cached dataset librispeech_asr_demo (C:/Users/olivi/.cache/huggingface/datasets/hf-internal-testing___librispeech_asr_demo/clean/2.1.0/d3bc4c2bc2078fcde3ad0f0f635862e4c0fef78ba94c4a34c4c250a097af240b)
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\olivi\\.cache\\huggingface\\datasets\\downloads\\extracted\\70070395e1a266c43ac19a8da1134801a8d0725b13c03e6f199937974534a2a2\\dev_clean\\1272\\128104\\1272-128104-0000.flac'

In [103]:
 from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
 from datasets import load_dataset
 import torch
 !pip install soundfile librosa

 # load model and tokenizer
 processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
 model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
     
 # load dummy dataset and read soundfiles
 ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")
 
 # tokenize
 input_values = processor(np.array(ds.features['audio']), return_tensors="pt", padding="longest").input_values  # Batch size 1
 
 # retrieve logits
 logits = model(input_values).logits
 
 # take argmax and decode
 predicted_ids = torch.argmax(logits, dim=-1)
 transcription = processor.batch_decode(predicted_ids)


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Found cached dataset librispeech_asr_dummy (C:/Users/olivi/.cache/huggingface/datasets/patrickvonplaten___librispeech_asr_dummy/clean/2.1.0/f2c70a4d03ab4410954901bde48c54b85ca1b7f9bf7d616e7e2a72b5ee6ddbfc)
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


TypeError: len() of unsized object